In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 89% 76.0M/85.7M [00:00<00:00, 127MB/s]
100% 85.7M/85.7M [00:00<00:00, 126MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 7.0 MB/s 
     |████████████████████████████████| 6.4 MB 73.3 MB/s 
     |████████████████████████████████| 840 kB 38.8 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 87 kB 8.2 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
     |████████████████████████████████| 120 kB 103.6 MB/s 
     |████████████████████████████████| 1.1 MB 72.9 MB/s 
     |████████████████████████████████| 234 kB 86.2 MB/s 
     |████████████████████████████████| 3.4 MB 64.7 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.1 MB/s 
     |████████████████████████████████| 99 kB 11.7 MB/s 
     |████████████████████████████████| 352 kB 66.7 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 462 kB 88.0 MB/s 
     |█████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [10]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [18]:
spec = model_spec.get('efficientdet_lite1')

In [19]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=30, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/30
397/397 [==============================] - 134s 256ms/step - det_loss: 1.0394 - cls_loss: 0.7836 - box_loss: 0.0051 - reg_l2_loss: 0.0717 - loss: 1.1112 - learning_rate: 0.0078 - gradient_norm: 3.3444 - val_det_loss: 0.5835 - val_cls_loss: 0.4841 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0720 - val_loss: 0.6556
Epoch 2/30
397/397 [==============================] - 100s 252ms/step - det_loss: 0.6513 - cls_loss: 0.5165 - box_loss: 0.0027 - reg_l2_loss: 0.0723 - loss: 0.7236 - learning_rate: 0.0074 - gradient_norm: 3.4657 - val_det_loss: 0.5538 - val_cls_loss: 0.4129 - val_box_loss: 0.0028 - val_reg_l2_loss: 0.0725 - val_loss: 0.6264
Epoch 3/30
397/397 [==============================] - 99s 250ms/step - det_loss: 0.5643 - cls_loss: 0.4580 - box_loss: 0.0021 - reg_l2_loss: 0.0727 - loss: 0.6370 - learning_rate: 0.0074 - gradient_norm: 3.0880 - val_det_loss: 0.4256 - val_cls_loss: 0.3655 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0728 - val_loss: 0.4985
Epoch 4/30
397/397 [===

In [20]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 3113.679764032364


In [21]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 8s 62ms/step

{'AP': 0.8115674, 'AP50': 0.9277198, 'AP75': 0.87656873, 'APs': -1.0, 'APm': 0.5777142, 'APl': 0.824583, 'ARmax1': 0.74710935, 'ARmax10': 0.86134857, 'ARmax100': 0.8708335, 'ARs': -1.0, 'ARm': 0.6788095, 'ARl': 0.88194776, 'AP_/10': 0.759351, 'AP_/20': 0.8240029, 'AP_/50': 0.8336428, 'AP_/100': 0.90833616, 'AP_/200': 0.8494571, 'AP_/500': 0.80042607, 'AP_/2000': 0.70575607}


In [22]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 10.523140668869019


In [16]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [ ]:
model.export(export_dir='/content/')